# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "output_data/cities.csv"
output_data_file
cities_df = pd.read_csv(output_data_file)
cities_df.drop(columns = "Unnamed: 0", inplace = True)
cities_df.head(1000)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hokitika,0,NZ,1599539044,69,-42.72,170.97,53.55,6.98
1,Busselton,85,AU,1599539045,71,-33.65,115.33,63.00,7.00
2,Kavieng,5,PG,1599539046,74,-2.57,150.80,84.81,11.01
3,Cidreira,100,BR,1599538501,92,-30.18,-50.21,58.55,17.34
4,Camacha,40,PT,1599538457,83,33.08,-16.33,71.60,13.87
...,...,...,...,...,...,...,...,...,...
581,Howard Springs,75,AU,1599539546,13,-12.50,131.05,97.00,12.75
582,Shahrud,63,IR,1599539547,18,36.42,54.98,79.23,5.39
583,São João da Barra,40,BR,1599539549,87,-21.64,-41.05,72.86,19.08
584,Paso de Carrasco,17,UY,1599539549,76,-34.86,-56.05,48.20,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Generate cities Lat and Lng as locations and Humidity as the weight.
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_df[["Lat", "Lng"]]

# locations.set_index('Lat').join(locations.set_index('Lng'))

locations

,Lat,Lng
0,-42.72,170.97
1,-33.65,115.33
2,-2.57,150.80
3,-30.18,-50.21
4,33.08,-16.33
...,...,...
581,-12.50,131.05
582,36.42,54.98
583,-21.64,-41.05
584,-34.86,-56.05


In [36]:
#Configure gmaps
fig_layout = {"withd": "2000px"
             ,"height": "1000px"
             ,"border": "1px solid black"
             ,"padding": "1px"
             ,"margin": "0 auto 0 auto"}

#Add Heatmap layer to map and plot
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False, 
                                 max_intensity=maxhumidity,
                                 point_radius=3, opacity = .5)

markers = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities to fit weather conditions.
narrowed_city_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Cloudiness"] == 0), :]
# Drop any rows will null values.
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,0,PF,1599539070,85,-23.12,-134.97,72.72,10.89
1,São Félix do Xingu,0,BR,1599539080,37,-6.64,-51.99,73.11,1.72
2,Guerrero Negro,0,MX,1599538910,70,27.98,-114.06,76.95,12.10
3,Sabha,0,LY,1599539140,26,27.04,14.43,78.94,9.46
4,Sumbe,0,AO,1599539149,86,-11.21,13.84,70.66,3.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
#Store into variable named hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

#Search for Hotels nearby
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Retrieving Results for Index 0: Rikitea.
Closest hotel in Rikitea is Pension Maro'i.
Retrieving Results for Index 1: São Félix do Xingu.
Closest hotel in São Félix do Xingu is Apart Hotel Rio Xingu.
Retrieving Results for Index 2: Guerrero Negro.
Closest hotel in Guerrero Negro is Hotel Los Caracoles.
Retrieving Results for Index 3: Sabha.
Closest hotel in Sabha is فندق البركولي.
Retrieving Results for Index 4: Sumbe.
Closest hotel in Sumbe is Hotel Ritz Sumbe.
Retrieving Results for Index 5: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is Hotel Mehmon Saroy.
Retrieving Results for Index 6: Nelson Bay.
Closest hotel in Nelson Bay is Amarna Luxury Beach Resort - Guest Only.
Retrieving Results for Index 7: Goiatuba.
Closest hotel in Goiatuba is Hotel Barrocos.
Retrieving Results for Index 8: Saravan.
Missing field/result... skipping.
Retrieving Results for Index 9: Jaciara.
Closest hotel in Jaciara is Hotel Toquinho.
Retrieving Results for Index 10: Koutsouras.
Closest hotel in Kout

,City,Country,Lat,Lng,Hotel Name
0,Rikitea,PF,-23.12,-134.97,Pension Maro'i
1,São Félix do Xingu,BR,-6.64,-51.99,Apart Hotel Rio Xingu
2,Guerrero Negro,MX,27.98,-114.06,Hotel Los Caracoles
3,Sabha,LY,27.04,14.43,فندق البركولي
4,Sumbe,AO,-11.21,13.84,Hotel Ritz Sumbe
5,Yangi Marg`ilon,UZ,40.43,71.72,Hotel Mehmon Saroy
6,Nelson Bay,AU,-32.72,152.15,Amarna Luxury Beach Resort - Guest Only
7,Goiatuba,BR,-18.01,-49.35,Hotel Barrocos
8,Saravan,AM,39.72,45.65,
9,Jaciara,BR,-15.97,-54.97,Hotel Toquinho


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]


In [33]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))